In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
conf = (SparkConf().setAppName('NoShuffleWithBroadcast').setMaster('yarn') \
    .set('spark.sql.adaptive.enabled', 'false'))

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

app_id                                   application_1727681258360_0044


In [4]:
# create DataFrame of length 1000 with 4 partitions with column 'val' 
partitions_num = 4
data = [ [k] for k in range(0, 1000) ] # transform to 2-D list
df = spark.sparkContext.parallelize(data, partitions_num).toDF(["val"])

# view partitions num we get when doing shuffle 
print("spark.sql.shuffle.partitions =", spark.conf.get("spark.sql.shuffle.partitions"))

print("df partitions num =", df.rdd.getNumPartitions()) # initial partitions num equals 4

spark.sql.shuffle.partitions = 200
df partitions num = 4


In [5]:
from pyspark.sql.functions import broadcast

df2 = df.withColumnRenamed('val', 'val2') # to avoid column names ambiguity when joining

# make common join 
df_joined = df.join(df2, df['val'] == df2['val2'], "inner")
print("df_joined partitions num =", df_joined.rdd.getNumPartitions()) 
# df_shuffled partitions num = 200 - there was shuffle

df_broadcasted = broadcast(df2) # make broadcasted df and join with it
df_joined_with_broacasted = df.join(df_broadcasted, df["val"] == df_broadcasted["val2"], "inner")
# df_joined_with_broacasted.count()
print("df_joined_with_broacasted partitions num =", df_joined_with_broacasted.rdd.getNumPartitions()) 
# df_joined partitions num = 4 - there was no shuffle

df_joined partitions num = 200


df_joined_with_broacasted partitions num = 4


In [6]:
spark.stop()